In [1]:
import os
import torch
from datasets import load_dataset, Dataset
import pandas as pd
import transformers
from transformers import AutoTokenizer
from trl import SFTTrainer
import transformers
from peft import AutoPeftModelForCausalLM
from transformers import GenerationConfig
from pynvml import *
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders import Docx2txtLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import glob
from ragatouille import RAGTrainer
from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders import Docx2txtLoader
from langchain_community.document_loaders import WebBaseLoader

In [2]:
# directory_path = r'/home/sdm/Desktop/shakib/KUET LLM/data/'
# file_pattern = '*.docx'  
# file_paths = glob.glob(directory_path + file_pattern)
# file_paths

In [3]:
# all_doc=[]
# for x in file_paths:
#     loader = Docx2txtLoader(x)
#     documents=loader.load()
#     # print(((documents)[0]).page_content)
#     all_doc.append(str(documents[0].page_content))
# docs='\n\n'.join(all_doc)
start=1
full_data=[]
while True:
    loader = WebBaseLoader(f"https://raw.githubusercontent.com/shahidul034/KUET-LLM-RAG-data/main/rag{start}.txt")
    data = loader.load()
    full_data.extend(data[0].page_content)
    start+=1
    if (data[0].page_content)=="404: Not Found":
        break
docs="\n\n".join(full_data)

In [4]:
from ragatouille import RAGPretrainedModel

RAG = RAGPretrainedModel.from_pretrained("colbert-ir/colbertv2.0")

In [5]:
RAG.index(
    collection=[docs],
    index_name="llama_colbert",
    max_document_length=256,
    split_documents=True,
)

---- WARNING! You are using PLAID with an experimental replacement for FAISS for greater compatibility ----
This is a behaviour change from RAGatouille 0.8.0 onwards.
This works fine for most users and smallish datasets, but can be considerably slower than FAISS and could cause worse results in some situations.
If you're confident with FAISS working on your machine, pass use_faiss=True to revert to the FAISS-using behaviour.
--------------------


[Mar 24, 21:40:23] #> Note: Output directory .ragatouille/colbert/indexes/llama_colbert already exists


[Mar 24, 21:40:23] #> Will delete 10 files already at .ragatouille/colbert/indexes/llama_colbert in 20 seconds...
[Mar 24, 21:40:45] [0] 		 #> Encoding 1760 passages..
[Mar 24, 21:40:47] [0] 		 avg_doclen_est = 92.62613677978516 	 len(local_sample) = 1,760
[Mar 24, 21:40:47] [0] 		 Creating 4,096 partitions.
[Mar 24, 21:40:47] [0] 		 *Estimated* 163,022 embeddings.
[Mar 24, 21:40:47] [0] 		 #> Saving the indexing plan to .ragatouille/colbe

0it [00:00, ?it/s]

[Mar 24, 21:40:48] [0] 		 #> Encoding 1760 passages..


1it [00:00,  1.08it/s]
100%|██████████| 1/1 [00:00<00:00, 199.34it/s]

[Mar 24, 21:40:49] #> Optimizing IVF to store map from centroids to list of pids..
[Mar 24, 21:40:49] #> Building the emb2pid mapping..
[Mar 24, 21:40:49] len(emb2pid) = 163022



100%|██████████| 4096/4096 [00:00<00:00, 274053.55it/s]

[Mar 24, 21:40:49] #> Saved optimized IVF to .ragatouille/colbert/indexes/llama_colbert/ivf.pid.pt
Done indexing!


'.ragatouille/colbert/indexes/llama_colbert'

In [6]:
retriever1 = RAG.as_langchain_retriever(k=3)

In [7]:

def create_vectorstore(flag):
    if flag == False:
        flag=True
        # loader = Docx2txtLoader(r"data/Khulna University of Engineering.docx")
        # documents=loader.load()
        all_doc=[]
        # for x in file_paths:
        #     loader = Docx2txtLoader(x)
        #     documents=loader.load()
        #     all_doc.extend(documents)
        # text_splitter=splitter_ret()
        from data_ret import data_ret
        all_doc=data_ret()
        text_splitter=CharacterTextSplitter(chunk_size=500,chunk_overlap=30,separator="\n")
        docs=text_splitter.split_documents(documents=all_doc)
        # text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
        # docs = text_splitter.split_documents(documents)
        model_name = "BAAI/bge-large-en-v1.5"
        hf = HuggingFaceEmbeddings(model_name=model_name)
        vectorstore=FAISS.from_documents(docs,hf)
        vectorstore.save_local('vectorstore')
        return vectorstore
    else:
        model_name = "BAAI/bge-large-en-v1.5"
        hf = HuggingFaceEmbeddings(model_name=model_name)
        new_vectorstore=FAISS.load_local("vectorstore",hf)
        return new_vectorstore

In [8]:
vectorstore=create_vectorstore(False)

Created a chunk of size 1517, which is longer than the specified 500
Created a chunk of size 629, which is longer than the specified 500
Created a chunk of size 666, which is longer than the specified 500
Created a chunk of size 1269, which is longer than the specified 500
Created a chunk of size 1100, which is longer than the specified 500
Created a chunk of size 556, which is longer than the specified 500
Created a chunk of size 604, which is longer than the specified 500
Created a chunk of size 658, which is longer than the specified 500
Created a chunk of size 710, which is longer than the specified 500
Created a chunk of size 699, which is longer than the specified 500
Created a chunk of size 720, which is longer than the specified 500
Created a chunk of size 672, which is longer than the specified 500
Created a chunk of size 731, which is longer than the specified 500
Created a chunk of size 1488, which is longer than the specified 500
Created a chunk of size 1231, which is longe

In [9]:
retriever2=vectorstore.as_retriever()

In [10]:
from langchain.retrievers import EnsembleRetriever
retriever = EnsembleRetriever(retrievers=[retriever2, retriever2],
                                            weights=[0.50, 0.50])

In [11]:
from transformers import pipeline
import transformers
base_model = "NousResearch/Llama-2-7b-chat-hf"
full_output = 'models/full_KUET_LLM_llama'
tokenizer = transformers.AutoTokenizer.from_pretrained(base_model)
# pipe = pipeline(task="text-generation", model=full_output, tokenizer=tokenizer, max_length=630)
pipe = pipeline("text-generation",
                model=full_output,
                tokenizer= tokenizer,
                torch_dtype=torch.bfloat16,
                device_map="auto",
                max_new_tokens = 512,
                do_sample=True,
                top_k=30,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id
                )
from langchain import HuggingFacePipeline
llm = HuggingFacePipeline(pipeline = pipe, model_kwargs = {'temperature':0})

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/home/sdm/anaconda3/envs/llmtesting/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/sdm/anaconda3/envs/llmtesting/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:415: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/sdm/anaconda3/envs/llm

In [12]:
from langchain import hub
prompt = hub.pull("rlm/rag-prompt")
prompt

ChatPromptTemplate(input_variables=['context', 'question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))])

In [13]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)
# torch.cuda.empty_cache()

In [14]:
def ans_ret(inp,history):
    ans=rag_chain.invoke(inp)
    k=ans.split("Based on the text material")
    k2=ans.split("Hope that helped! Let me know if you have any more questions.")

    if len(k)>=2:
        k3=k[0].split("Hope that helped! Let me know if you have any more questions.")
        if len(k3)>=2:
            return k3[0]
        else:
            return k[0]
    if len(k2)>=2:
        return k2[0]
    return ans

In [15]:
import pandas as pd
import tqdm
dat=[]
cnt=0
df=pd.read_excel(r"data/KUET information2k20.xlsx")
df.head()
for id,ques in tqdm.tqdm(zip(df['id'],df['Question'])):
    ans=ans_ret(ques,"")
    dat.append({
        "id":id,
        "question":ques,
        "answer":ans
    })
    # if cnt==5:
    #     break
    # cnt+=1
    

10it [00:09,  1.00s/it]/home/sdm/anaconda3/envs/llmtesting/lib/python3.9/site-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
11it [00:10,  1.05s/it]/home/sdm/anaconda3/envs/llmtesting/lib/python3.9/site-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
12it [00:10,  1.17it/s]/home/sdm/anaconda3/envs/llmtesting/lib/python3.9/site-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
13it [00:11,  1.24it/s]/home/sdm/anaconda3/envs/llmtesting/lib/python3.9/site-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximi

In [16]:
sa=pd.DataFrame(dat)
model_ans="model_ans_llama_finetuned486_rag_ensemble"
sa.to_excel(f"data//{model_ans}.xlsx",index=False)